In [1]:
!bash /home/azureuser/cloudfiles/code/blobfuse/blobfuse_raadsinformatie.sh

In [25]:
import sys
sys.path.append("..")

# Select where to run notebook: "azure" or "local"
my_run = "azure"

import my_secrets as sc
import settings as st

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf

### Goal: remove faulty documents
The goal is to remove faulty documents where mistakes were made when converting pdf into txt.

In [27]:
# load data
import ast
import pandas as pd

txtfiles = pd.read_pickle(f"{cf.output_path}/txtfiles_notcleaned.pkl")
# txtfiles['tokens'] = txtfiles['tokens'].apply(ast.literal_eval)
# display(txtfiles.head())
df = txtfiles.copy()
# df['clean_text'] = df['clean_tokens'].apply(lambda x: ' '.join(x))


In [29]:
display(df)
print(len(set(df['label'])))
display(df.groupby('label')['token_count'].describe())

,label,path,id,set,text,tokens,token_count,clean_tokens,clean_tokens_count,pdf_path,num_pages
0,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,0,train,Gemeente Amsterdam\n% Gemeenteraad R\n% Gemeen...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",395,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",205,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0
1,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,1,train,Gemeente Amsterdam\n\n% Gemeenteraad R\n\n% Ge...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",390,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",197,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0
2,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,2,train,Gemeente Amsterdam\n\n% Gemeenteraad R\n\n% Ge...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",389,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",192,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0
3,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,3,train,Gemeente Amsterdam\n\n% Gemeenteraad R\n\n% Ge...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",464,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",225,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0
4,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,4,test,x Gemeente Amsterdam R\nGemeenteraad\n\n% Geme...,"[x, Gemeente, Amsterdam, R, Gemeenteraad, %, G...",261,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",134,/home/azureuser/cloudfiles/code/blobfuse/raads...,1.0
...,...,...,...,...,...,...,...,...,...,...,...
33112,Factsheets,/home/azureuser/cloudfiles/code/blobfuse/raads...,33123,train,"In Amsterdam is, net als in andere delen van h...","[In, Amsterdam, is, ,, net, als, in, andere, d...",3816,"[Amsterdam, net, delen, land, sprake, tekort, ...",2001,/home/azureuser/cloudfiles/code/blobfuse/raads...,10.0
33113,Factsheets,/home/azureuser/cloudfiles/code/blobfuse/raads...,33124,val,| September 2021 C 1 / )\nr 1E | rs | a 4 in\n...,"[|, September, 2021, C, 1, /, ), r, 1E, |, rs,...",85228,"[September, 2021, 1E, rs, ke, We, ear, nrc, TW...",43467,/home/azureuser/cloudfiles/code/blobfuse/raads...,164.0
33114,Factsheets,/home/azureuser/cloudfiles/code/blobfuse/raads...,33125,train,WPI KWARTAAL FACTSHEET DECEMBER 2022\nKERNCIJF...,"[WPI, KWARTAAL, FACTSHEET, DECEMBER, 2022, KER...",1595,"[WPI, KWARTAAL, FACTSHEET, DECEMBER, 2022, KER...",970,/home/azureuser/cloudfiles/code/blobfuse/raads...,7.0
33115,Factsheets,/home/azureuser/cloudfiles/code/blobfuse/raads...,33126,train,"WPI VOORTGANGSRAPPORTAGE\nKERNCIJFERS WERK, PA...","[WPI, VOORTGANGSRAPPORTAGE, KERNCIJFERS, WERK,...",1821,"[WPI, VOORTGANGSRAPPORTAGE, KERNCIJFERS, WERK,...",1024,/home/azureuser/cloudfiles/code/blobfuse/raads...,6.0


15


,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
Actualiteit,996.0,696.773092,3462.293848,72.0,236.00,344.5,592.0,103700.0
Adviesaanvraag,2442.0,1644.871417,2993.302964,57.0,557.00,955.0,1672.0,77769.0
Agenda,3537.0,1048.904156,4244.783305,20.0,325.00,616.0,939.0,139648.0
Amendement,1969.0,2658.493652,7044.470410,30.0,236.00,359.0,1184.0,62304.0
Begroting,1967.0,13160.190646,46780.793626,40.0,248.00,385.0,2324.0,247184.0
Besluit,775.0,986.649032,1749.006500,70.0,168.50,439.0,1216.0,26523.0
Brief,1995.0,1764.259649,1728.612069,3.0,734.00,1269.0,2291.5,32957.0
Factsheets,234.0,6008.987179,14742.837685,112.0,1154.00,2753.5,5462.0,171297.0
Motie,8336.0,521.707893,1129.571368,105.0,234.00,292.5,402.0,36091.0


In [31]:
yeet = pd.read_pickle(f"{cf.output_path}/txtfiles.pkl")
print(len(set(yeet['label'])))
from collections import Counter
print(Counter(yeet['label']))


12
Counter({'Motie': 8336, 'Agenda': 3537, 'Voordracht': 3332, 'Schriftelijke Vragen': 2974, 'Brief': 1992, 'Raadsadres': 1973, 'Onderzoeksrapport': 1286, 'Actualiteit': 996, 'Termijnagenda': 996, 'Besluit': 775, 'Raadsnotulen': 273, 'Factsheets': 234})


In [135]:
display(df.groupby('label')['clean_tokens_count'].describe())

,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
Actualiteit,996.0,360.197791,1833.846209,39.0,124.00,175.0,311.25,55400.0
Adviesaanvraag,2442.0,825.560197,1517.594501,33.0,287.00,486.0,818.00,39865.0
Agenda,3537.0,559.212327,2187.277067,17.0,180.00,337.0,515.00,72471.0
Amendement,1969.0,1274.853225,3332.397979,15.0,118.00,179.0,595.00,29968.0
Begroting,1967.0,7290.902898,26004.241417,24.0,125.00,203.0,1179.00,137501.0
Besluit,775.0,513.227097,889.155749,50.0,95.50,230.0,657.00,13651.0
Brief,1995.0,881.689724,954.436225,3.0,369.00,636.0,1134.00,21285.0
Factsheets,234.0,3347.893162,9863.108583,60.0,634.75,1527.0,2937.00,131816.0
Motie,8336.0,264.074976,584.938299,60.0,120.00,149.0,204.00,21285.0


Remove the faulty documents. The rest of the notebook explain why they are removed.

In [121]:
# remove Adviesaanvraag, Amendement and Begroting. These classes include docs from other classes.
df = df.loc[~df['label'].isin(['Adviesaanvraag', 'Amendement', 'Begroting'])]

# remove docs that are too short (only include gibberish)
subdf = df.loc[df['label']== 'Schriftelijke Vragen']
threshold = subdf['clean_tokens_count'].quantile(0.01)
subdf = subdf.loc[df['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
df = df.loc[~df['id'].isin(subdf['id'])]

subdf = df.loc[(df['label']== 'Brief') & (df['clean_tokens_count'] < 75)]
df = df.loc[~df['id'].isin(subdf['id'])]

subdf = df.loc[(df['label']== 'Raadsadres') & (df['clean_tokens_count'] < 25)]
df = df.loc[~df['id'].isin(subdf['id'])]

# rename verslag column
df.loc[df['label']=='Verslag','label'] = 'Raadsnotulen'

# safe
# df.to_pickle("f{cf.output_path}/txtfiles.pkl")

##### Remove image gibberish
images results in loads of gibberish (ususally tokens of 1,2,3 or 4 characters long). Check if tokens with less than 5 characters are a  dutch words, else remove the token. 


In [73]:
# import nltk
# nltk.download('words')

# from nltk.corpus import words
# dutch_words = set(words.words('nl'))

# def half_short_tokens(tokens):
#     num_short_tokens = sum(1 for token in tokens if len(token) <= 2)
#     return num_short_tokens >= len(tokens) / 2

# df['image'] = df['clean_tokens'].apply(half_short_tokens)

# def remove_gibberish(tokens):
#     clean_tokens = []
#     for token in tokens:
#         if token.lower() in dutch_words:
#             clean_tokens.append(token)

#     return clean_tokens

# df['no_gibberish_tokens'] = df['clean_tokens'].apply(remove_gibberish)



##### Check if text if extracted from image (for example a map)
This text will just be gibberish and we want to filter those docs out. We do this by removing documents of which a half or more of the tokens have a length of 2 or less (so either one or two characters).

In [124]:
from collections import Counter

def half_short_tokens(tokens):
    num_short_tokens = sum(1 for token in tokens if len(token) <= 2)
    return num_short_tokens >= len(tokens) / 2

df['image'] = df['clean_tokens'].apply(half_short_tokens)

# for index, row in df.loc[df['image']==True].iterrows():
#     print(row['label'], row['clean_tokens_count'], row['path'])
#     print(row['clean_tokens'])
#     print('\n\n')
    
# print(Counter(df.loc[df['image']==True]['label']))
# display(df.loc[df['image']==True])
# print(df.loc[df['image']==True].iloc[0]['text'])

##### Average tokens per page

In [125]:
# df['tokens_page'] = df['clean_tokens_count']/df['num_pages']
# display(df['tokens_page'].describe())
# class_describe = df.groupby('label')['tokens_page'].describe()
# display(class_describe)
# display(df.head())

##### Actualiteit
After checking the lowest 1 percent of number of tokens (10 documents which have less than 58 tokens) were checked using the PDFs. These documents are just very short, thus not a mistake of the txt extraction. 

In [7]:
actualiteit = df.loc[df['label']=='Actualiteit']
threshold = actualiteit['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = actualiteit[actualiteit['clean_tokens_count'] < threshold]
# actualiteit['clean_tokens_count'].describe()


##### Adviesaanvraag
Txt extraction from PDFs went well, except for image. The images result in gibberish in the data. Additionally, it does not seem like there are only adviesaanvragen in there. 

In [57]:
adviesaanvraag = df.loc[df['label']=='Adviesaanvraag']
threshold = adviesaanvraag['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = adviesaanvraag[adviesaanvraag['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# adviesaanvraag['clean_tokens_count'].describe()


#### Agenda
Agenda's can just be very short. Txt extraction went well.

In [126]:
subdf = df.loc[df['label']=='Agenda']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


#### Amendement
Not only amendement docs are included. txt extraction went well.

In [127]:
subdf = df.loc[df['label']=='Amendement']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])

# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


#### Begroting

It does not seem like there are only begrotingen. txt went well. except for tables (file:///C:/Users/femke/Documents/MasterThesis/discardfiles/1543430.pdf)

In [63]:
subdf = df.loc[df['label']=='Begroting']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


In [87]:
# subdf = df.loc[df['label']== 'Begroting']
# threshold = subdf['clean_tokens_count'].quantile(0.49)
# threshold2 = subdf['clean_tokens_count'].quantile(0.50)

# lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# lower_5_percentile_rows = subdf[(subdf['clean_tokens_count'] < threshold2) & (subdf['clean_tokens_count']>threshold)].sort_values(by=['clean_tokens_count'])

# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()



#### Besluit
Looks good

In [21]:
subdf = df.loc[df['label']=='Besluit']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


#### Brief
Remove docs with less than 75 tokens. These are some weird poster. The rest looks good.

In [65]:
subdf = df.loc[df['label']=='Brief']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


#### Factsheets
Lots of posters. Lots of images included

In [79]:
subdf = df.loc[df['label']=='Factsheets']
threshold = subdf['clean_tokens_count'].quantile(0.1)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
print(len(lower_5_percentile_rows))

for index, row in lower_5_percentile_rows.iterrows():
    print(row['clean_tokens_count'], row['path'])
    print(row['text'])
    print('\n\n')

subdf['clean_tokens_count'].describe()


24
60 /home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/OpenResearch/factsheets/factsheet-vves.pdf.ocr
ENERGIETRANSITIE BEWONERSVERENIGINGEN
AMSTERDAM-ZUIDOOST |
n Ee 0
Hoe kunnen bewonersverenigingen
An gestimuleerd worden om mee te doen \ )
aan de groene energie transitie? I__
BELEMMERINGEN
Belangenverschil huur Ongeschikte Tekort aan geld
en koop woningen Infrastructuur zoals _ annis & vertrouwen
(lange vs korte termijn) kabels en daken
' 9) MOGELIJKHEDEN
| & Ü en Jy ON Bewoners kunnen door
| nn ON deze dingen gestimuleerd
| en ' worden
Geld Kennis Vertrouwen
e Subsidies & duidelijke geldstromen as
waar bewoners van afweten ee ‚
e Praktische kennis en begeleiding 7, [ dn
e Overheid, gemeente en initatieven die
zich verplaatsen in de bewoner Ii
‚LL




78 /home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/OpenResearch/factsheets/factsheet-bewonerscommissies.pdf.ocr
Er bestaat frictie tussen de bewoners(commissie) en ©
Eigen haard door tegenstrijdige aandachtspunten © A


count       234.000000
mean       3347.893162
std        9863.108583
min          60.000000
25%         634.750000
50%        1527.000000
75%        2937.000000
max      131816.000000
Name: clean_tokens_count, dtype: float64

#### Motie
I cant see the original PDFs. Looks good tho.

In [55]:
subdf = df.loc[df['label']=='Motie']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


#### Onderzoeksrapport      
Include posters, presentations? (/home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/OpenResearch/onderzoek-in-de-gemeenteraad/onderzoeken-rapporten-2019-gemeenteraad/5g-technische-sessie.pdf.ocr
)


Includes images.

In [66]:
subdf = df.loc[df['label']=='Onderzoeksrapport']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
print(len(lower_5_percentile_rows))

for index, row in lower_5_percentile_rows.iterrows():
    print(row['clean_tokens_count'], row['path'])
    print(row['text'])
    print('\n\n')

subdf['clean_tokens_count'].describe()


13
162 /home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/OpenResearch/onderzoek-in-de-gemeenteraad/onderzoeken-rapporten-2022-gemeenteraad/infographic-amsterdamse-bos---duurzaam-op-orde.pdf.ocr
Amsterdamse Bos duurzaam op orde
A. Actueel inzicht in beheer en onderhoud
Achterstallig onderhoud x 1.000 Regulier budget x 1.000
12.000 12.100
LACE Tekort
VA VOR Tekort
GRON Beschikbaar EMI 8 Beschikbaar
Groen Bruggen Spelen & Water __Beschoeiingen Dagelijks Groot Vervangings- Capaciteit
meubilair Onderhoud Onderhoud onderhoud
Ingroeimodel voor benodigd extra budget x 1.000
SHUT Dagelijks beheer en onderhoud
2.500 Groot onderhoud
2.000 Achterstallig onderhoud
„soo | Cepeciteit regie en organisatie
1.000
Ì | | |
0 ll mall Il m H
2022 [2023 [ao2a [2025 | 2026 | 2027 | 2028 | 2029 [2030 |
Jaarlijkse opbouw naar benodigd regulier budget x 1.000
€ 389 €0
7.000 . € 440 ]
Nn Aanvraag per jaar” € 2.221
6.000
Verwachte extra eigen inkomsten
5.000
Re €2131 WM
3.000
€ 363
2.000 € 514
1.000 CM
0 €0

count      1286.000000
mean       7970.667963
std        9994.442762
min         162.000000
25%        3049.250000
50%        5631.500000
75%        9927.250000
max      143782.000000
Name: clean_tokens_count, dtype: float64

#### Raadsadres
remove less than 25 tokens. looks good.

In [68]:
subdf = df.loc[df['label']=='Raadsadres']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


#### Schriftelijke Vragen
Cant open PDFs. Looks like a mess. Middle looks much better. Should remove lowest 0.01 cleaned_tokens_count

In [46]:
subdf = df.loc[df['label']== 'Schriftelijke Vragen']
threshold = subdf['clean_tokens_count'].quantile(0.01)
# threshold2 = subdf['clean_tokens_count'].quantile(0.02)

lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# lower_5_percentile_rows = subdf[(subdf['clean_tokens_count'] < threshold2) & (subdf['clean_tokens_count']>threshold)].sort_values(by=['clean_tokens_count'])

# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()



#### Termijnagenda
Does include many tables.

In [48]:
subdf = df.loc[df['label']=='Termijnagenda']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


#### Verslag
Should rename into raadsnotulen. looks good

In [51]:
subdf = df.loc[df['label']=='Verslag']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


#### Voordracht
Looks good

In [54]:
subdf = df.loc[df['label']=='Voordracht']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()
